In [1]:
import cv2
from xgboost import XGBClassifier
import os
import numpy as np
from cv2.xfeatures2d import SIFT_create as sift_create
from PIL import Image
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder

# Define paths and hyperparameters
data_path = "train-dataset"
max_descriptor_length = 0  # Initialize to 0

# Load image paths and labels
images = []
labels = []
for folder_name in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder_name)
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        label = folder_name  # Assuming folder name represents class label
        images.append(image_path)
        labels.append(label)
# Encode labels as integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_encoded = labels_encoded.astype(np.int32)  # Convert to np.int32


def extract_surf_features(image_path):
    global max_descriptor_length
    img = Image.open(image_path)
    img = img.resize((512, 512))  # Resize the image
    img = np.array(img)  # Convert PIL Image to NumPy array
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # Convert to grayscale
    surf = sift_create()
    keypoints, descriptors = surf.detectAndCompute(gray, None)
    descriptors = descriptors.flatten()
    max_descriptor_length = max(max_descriptor_length, len(descriptors))
    return descriptors

# Extract features from training images and find the maximum descriptor length
for image_path in images:
    extract_surf_features(image_path)

# Pad or truncate the descriptors to match the maximum length
image_features = []
for image_path in images:
    features = extract_surf_features(image_path)
    padded_features = np.pad(features, (0, max_descriptor_length - len(features)), mode='constant')
    image_features.append(padded_features)
# Train the XGBClassifier
clf = XGBClassifier(random_state=42)
clf.fit(np.array(image_features), labels_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [2]:
def classify_image(image_path, clf, label_encoder):
    features = extract_surf_features(image_path)
    padded_features = np.pad(features, (0, max_descriptor_length - len(features)), mode='constant')
    predicted_class_idx = clf.predict([padded_features])[0]
    predicted_class = label_encoder.inverse_transform([predicted_class_idx])[0]
    return predicted_class

# Test on new images and calculate accuracy
wrong = 0
total = 0
for i in range(100):
    for image_path, label in zip(images, labels):
        predicted_class = classify_image(image_path, clf, label_encoder)
        if predicted_class != label:
            wrong += 1
        total += 1
    print (total)

accuracy = (1 - (wrong / total)) * 100
print(f"Errors: {wrong}")
print(f"Accuracy: {accuracy:.2f}%")

52
104
156
208
260
312
364
416
468
520
572
624
676
728
780
832
884
936
988
1040
1092
1144
1196
1248
1300
1352
1404
1456
1508
1560
1612
1664
1716
1768
1820
1872
1924
1976
2028
2080
2132
2184
2236
2288
2340
2392
2444
2496
2548
2600
2652
2704
2756
2808
2860
2912
2964
3016
3068
3120
3172
3224
3276
3328
3380
3432
3484
3536
3588
3640
3692
3744
3796
3848
3900
3952
4004
4056
4108
4160
4212
4264
4316
4368
4420
4472
4524
4576
4628
4680
4732
4784
4836
4888
4940
4992
5044
5096
5148
5200
Errors: 0
Accuracy: 100.00%
